In [1]:
import pandas
import numpy as np
from pandas.plotting import scatter_matrix
import pandas
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

## Get data 

In [3]:
train = "D:/4-1/Thesis/conv/1525590021096_C_train.csv"
def load_train_set():
    return pandas.read_csv(train)

In [4]:
train_set = load_train_set()
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 10 columns):
Clump-Thickness                698 non-null int64
Uniformity-of-Cell-Size        698 non-null int64
Uniformity-of-Cell-Shape       698 non-null int64
Marginal-Adhesion              698 non-null int64
Single-Epithelial-Cell-Size    698 non-null int64
Bare-Nuclei                    698 non-null object
Bland-Chromatin                698 non-null int64
Normal-Nucleoli                698 non-null int64
Mitoses                        698 non-null int64
Class                          698 non-null int64
dtypes: int64(9), object(1)
memory usage: 54.6+ KB


In [5]:
train_set.head()

,Clump-Thickness,Uniformity-of-Cell-Size,Uniformity-of-Cell-Shape,Marginal-Adhesion,Single-Epithelial-Cell-Size,Bare-Nuclei,Bland-Chromatin,Normal-Nucleoli,Mitoses,Class
0,5,1,1,1,2,\t1\t,3,1,1,2
1,5,4,4,5,7,\t10\t,3,2,1,2
2,3,1,1,1,2,\t2\t,3,1,1,2
3,6,8,8,1,3,\t4\t,3,7,1,2
4,4,1,1,3,2,\t1\t,3,1,1,2


## correlation of target(class) with other attributes which attribute is more impactful

In [6]:
corr_matrix = train_set.corr()
corr_matrix["Class"].sort_values(ascending = False)

Class                          1.000000
Uniformity-of-Cell-Shape       0.818387
Uniformity-of-Cell-Size        0.817356
Bland-Chromatin                0.756262
Clump-Thickness                0.717283
Normal-Nucleoli                0.712551
Marginal-Adhesion              0.696525
Single-Epithelial-Cell-Size    0.683077
Mitoses                        0.424464
Name: Class, dtype: float64

## Attribute Bare-Nuclei is only object type data in this dataset, for scaling purpose need to change it to int  

In [7]:
bare = train_set["Bare-Nuclei"]
bare.value_counts()

\t1\t     402
\t10\t    132
\t2\t      30
\t5\t      29
\t3\t      28
\t8\t      21
\t4\t      19
\t?\t      16
\t9\t       9
\t7\t       8
\t6\t       4
Name: Bare-Nuclei, dtype: int64

## fixing Attribute Bare-Nuclei column 

In [8]:
sz = int(bare.size)
tr_bare = []
for i in range(sz):
    if(bare[i][1] == '?'):   # have some missing values denote by '?' in cells
        tr_bare.append(0)
    else:
        tr_bare.append(int(bare[i]))
        
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 10 columns):
Clump-Thickness                698 non-null int64
Uniformity-of-Cell-Size        698 non-null int64
Uniformity-of-Cell-Shape       698 non-null int64
Marginal-Adhesion              698 non-null int64
Single-Epithelial-Cell-Size    698 non-null int64
Bare-Nuclei                    698 non-null object
Bland-Chromatin                698 non-null int64
Normal-Nucleoli                698 non-null int64
Mitoses                        698 non-null int64
Class                          698 non-null int64
dtypes: int64(9), object(1)
memory usage: 54.6+ KB


In [9]:
tr_train_set= train_set.copy() #copy to set the original one aside
tr_train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 10 columns):
Clump-Thickness                698 non-null int64
Uniformity-of-Cell-Size        698 non-null int64
Uniformity-of-Cell-Shape       698 non-null int64
Marginal-Adhesion              698 non-null int64
Single-Epithelial-Cell-Size    698 non-null int64
Bare-Nuclei                    698 non-null object
Bland-Chromatin                698 non-null int64
Normal-Nucleoli                698 non-null int64
Mitoses                        698 non-null int64
Class                          698 non-null int64
dtypes: int64(9), object(1)
memory usage: 54.6+ KB


In [10]:
tr_train_set.drop(['Bare-Nuclei'], axis = 1, inplace= True)

In [11]:
tr_train_set['Bare-Nuclei'] = tr_bare

## Look at the Bare-Nuclei column, the data's converted to int64 now 

In [12]:
median = tr_train_set['Bare-Nuclei'].median()
tr_train_set['Bare-Nuclei'].fillna(median)
tr_train_set.head()

,Clump-Thickness,Uniformity-of-Cell-Size,Uniformity-of-Cell-Shape,Marginal-Adhesion,Single-Epithelial-Cell-Size,Bland-Chromatin,Normal-Nucleoli,Mitoses,Class,Bare-Nuclei
0,5,1,1,1,2,3,1,1,2,1
1,5,4,4,5,7,3,2,1,2,10
2,3,1,1,1,2,3,1,1,2,2
3,6,8,8,1,3,3,7,1,2,4
4,4,1,1,3,2,3,1,1,2,1


In [13]:
for i in range(sz):
    if(tr_train_set['Bare-Nuclei'][i] == 0):
         tr_train_set['Bare-Nuclei'][i] = median

In [14]:
tr_train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 10 columns):
Clump-Thickness                698 non-null int64
Uniformity-of-Cell-Size        698 non-null int64
Uniformity-of-Cell-Shape       698 non-null int64
Marginal-Adhesion              698 non-null int64
Single-Epithelial-Cell-Size    698 non-null int64
Bland-Chromatin                698 non-null int64
Normal-Nucleoli                698 non-null int64
Mitoses                        698 non-null int64
Class                          698 non-null int64
Bare-Nuclei                    698 non-null int64
dtypes: int64(10)
memory usage: 54.6 KB


In [15]:
tr_train_set.head()

,Clump-Thickness,Uniformity-of-Cell-Size,Uniformity-of-Cell-Shape,Marginal-Adhesion,Single-Epithelial-Cell-Size,Bland-Chromatin,Normal-Nucleoli,Mitoses,Class,Bare-Nuclei
0,5,1,1,1,2,3,1,1,2,1
1,5,4,4,5,7,3,2,1,2,10
2,3,1,1,1,2,3,1,1,2,2
3,6,8,8,1,3,3,7,1,2,4
4,4,1,1,3,2,3,1,1,2,1


## checking for missing values 

In [16]:
tr_train_set[tr_train_set == '?'].sum()

Clump-Thickness                0.0
Uniformity-of-Cell-Size        0.0
Uniformity-of-Cell-Shape       0.0
Marginal-Adhesion              0.0
Single-Epithelial-Cell-Size    0.0
Bland-Chromatin                0.0
Normal-Nucleoli                0.0
Mitoses                        0.0
Class                          0.0
Bare-Nuclei                    0.0
dtype: float64

In [17]:
corr_matrix = tr_train_set.corr()
corr_matrix['Class'].sort_values(ascending = False)

Class                          1.000000
Bare-Nuclei                    0.819361
Uniformity-of-Cell-Shape       0.818387
Uniformity-of-Cell-Size        0.817356
Bland-Chromatin                0.756262
Clump-Thickness                0.717283
Normal-Nucleoli                0.712551
Marginal-Adhesion              0.696525
Single-Epithelial-Cell-Size    0.683077
Mitoses                        0.424464
Name: Class, dtype: float64

## the following code will create stratified test set 

In [18]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(tr_train_set, tr_train_set["Bare-Nuclei"]):
    strat_train_set = tr_train_set.loc[train_index]
    strat_test_set = tr_train_set.loc[test_index]

In [19]:
strat_test_set['Bare-Nuclei'].value_counts()/len(strat_test_set)

1     0.600000
10    0.185714
5     0.042857
3     0.042857
2     0.042857
8     0.028571
4     0.028571
9     0.014286
7     0.007143
6     0.007143
Name: Bare-Nuclei, dtype: float64

In [20]:
tr_train_set['Bare-Nuclei'].value_counts()/len(tr_train_set)

1     0.598854
10    0.189112
2     0.042980
5     0.041547
3     0.040115
8     0.030086
4     0.027221
9     0.012894
7     0.011461
6     0.005731
Name: Bare-Nuclei, dtype: float64

In [21]:
Y_train = strat_train_set['Class']
Y_test = strat_test_set['Class']
X_train = strat_train_set.copy()
X_test = strat_test_set.copy()

In [22]:
X_test.drop(['Class'], axis = 1, inplace = True)

In [23]:
X_train.drop(['Class'], axis = 1, inplace = True)

# train a model and get accuracy score 

In [24]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
cancer_prediction = lin_reg.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score
reg_accuracy_score = accuracy_score(Y_test, cancer_prediction)*100
print('Accuracy score of LinerRegression: ',reg_accuracy_score, "%")

Accuracy score of LinerRegression:  50.0 %


In [28]:
nb_model = GaussianNB()
nb_model.fit(X_train, Y_train)

GaussianNB(priors=None)

In [29]:
predict = nb_model.predict(X_test)
nb_accuracy_score = accuracy_score(Y_test, predict)*100
print('Accuracy score of Naive Bayes: ',nb_accuracy_score,"%")

Accuracy score of Naive Bayes:  97.85714285714285 %


In [30]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state = 0)
tree_reg.fit(X_train, Y_train)
tree_predict = tree_reg.predict(X_test)
tree_reg_accuracy_score = accuracy_score(Y_test, tree_predict)*100
print('Accuracy score of Decision Tree Regressor: ',tree_reg_accuracy_score,"%")

Accuracy score of Decision Tree Regressor:  96.42857142857143 %


## after deleting less impactfull attribute 'Mitoses' and train model 

In [31]:
# let's see after deleting mitosis 
combined = [X_train, X_test]
for set in combined: 
    set.drop(['Mitoses'], axis = 1, inplace = True)

In [33]:
lin_reg.fit(X_train, Y_train)
cancer_prediction = lin_reg.predict(X_test).astype(int)
reg_accuracy_score = accuracy_score(Y_test, cancer_prediction)*100
print('Accuracy score of LinerRegression: ',reg_accuracy_score, "%")

Accuracy score of LinerRegression:  49.28571428571429 %


In [34]:
model = GaussianNB()
model.fit(X_train, Y_train)
predict = model.predict(X_test)
print('Accuracy score of Naive Bayes: ',accuracy_score(Y_test, predict)*100,"%")

Accuracy score of Naive Bayes:  97.85714285714285 %


In [35]:
tree_reg = DecisionTreeRegressor(random_state = 0)
tree_reg.fit(X_train, Y_train)
tree_predict = tree_reg.predict(X_test)
print('Accuracy score of Decision Tree Regressior: ',accuracy_score(Y_test, tree_predict)*100,"%")

Accuracy score of Decision Tree Regressior:  96.42857142857143 %
